1. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Kérdezzük le, hogy melyik ügyfél (USERNEV) hány különböző szálláshelyen foglalt!</span>

a. A listában azok az ügyfelek is jelenjenek meg, akiknek még nem volt foglalásuk

b. Megfelelően jelölve jelenjen meg a végösszeg is!

In [ ]:
SELECT IIF(v.usernev IS NULL, 'Végösszeg', v.usernev) AS 'Ügyfél',
       COUNT(DISTINCT(szh.SZALLAS_ID)) AS 'Különböző szálláshelyek száma'
FROM Vendeg v LEFT JOIN Foglalas f ON v.USERNEV = f.UGYFEL_FK
              LEFT JOIN Szoba sz ON f.SZOBA_FK = sz.SZOBA_ID
              LEFT JOIN Szallashely szh ON sz.SZALLAS_FK = szh.SZALLAS_ID
GROUP BY ROLLUP(v.USERNEV)


2. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk listát, amely megjeleníti a vendégek adatait!</span>

- Egy új oszlopban számoljuk ki a vendég életkorát (években)
- Egy másik új oszlopban határozzuk meg, hogy születési dátum szerint növekvő rendezésnél mennyi az adott ügyfél. az előtte lévő 2 ügyfél és az utána lévő 2 ügyfél átlagos életkora! Az oszlopot nevezzük el értelemszerűen!

In [ ]:
SELECT *,
       DATEDIFF(YEAR, szul_dat, GETDATE()) AS 'Életkor',
       AVG(DATEDIFF(YEAR, szul_dat, GETDATE()))
           OVER(ORDER BY szul_dat ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING) AS 'Átlagos életkor'
FROM Vendeg

3. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Egészítsük ki a megkezdett lekérdezést, amely listázza azon vendégek azonosítóját és nevét, akik már legalább egyszer foglaltak, és MINDEN ESETBEN összesen két fő számára (felnőtt + gyermek szám összege)!&nbsp;</span> 

 a. A lista ne tartalmazzon ismétlődő sorokat!

In [ ]:
SELECT v.USERNEV, v.NEV
FROM Vendeg v JOIN Foglalas f ON v.USERNEV = f.UGYFEL_FK 
WHERE NOT EXISTS
( 
  SELECT *
  FROM Foglalas f2 
  WHERE v.USERNEV = f2.UGYFEL_FK
 AND f2.FELNOTT_SZAM + f2.GYERMEK_SZAM <>2
)